#I/ OVERVIEW
Phần này sẽ thử scrape data theo 2 cách để **so sánh** và chọn cách **tốt hơn** trước khi đi vào thực xử lý hàng loạt:

- Cách 1: Sử dụng module `BeautifulSoup` của library `bs4`.
- Cách 2: Sử dụng method `read_html` của library `pandas`.

Website sẽ dùng để tham khảo là:
[24hmoney.vn/ DS Cty HOSE](https://24hmoney.vn/companies?industry_code=all&floor_code=HOSE&com_type=all&letter=all&page=1)
gồm có 30 trang chứa thông tin như Mã Cổ Phiếu, Tên Công ty, Ngành ... của các Công ty đại chúng đang niêm yết trên sàn HOSE (Sở Giao dịch Chứng khoán Thành phố Hồ Chí Minh).

In [ ]:
# import library để thao tác với data dạng table
import pandas as pd

#II/ SCRAPING TESTING

##1/ Cách 1: Dùng `BeautifulSoup`.

### Step 1: Lấy content của website.


In [ ]:
# import lib để thực hiện tương tác với website cần scrape
import requests

# địa chỉ website (chỉ thực hiện với trang số 1 trong tổng số 30)
url = "https://24hmoney.vn/companies?industry_code=all&floor_code=HOSE&com_type=all&letter=all&page=1"

# thực hiện gửi yêu cầu và nhận phản hồi từ website
response = requests.get(url)

# lấy nội dung website về lưu vào 1 biến
if response.status_code == 200:
  html_content = response.content

### Step 2: Lấy data cần thiết từ content.
Đầu tiên là tên các column.

In [ ]:
# import module 'BeautifulSoup' từ library 'bs4'
from bs4 import BeautifulSoup

# khởi tạo trình phân tích đọc data từ html
soup = BeautifulSoup(html_content, "html.parser")

# tìm ra table cần lấy data từ class
div_table = soup.find("div", {"class": "vue-table"})

# lọc ra tên các column từ tag 'thead'
table_head = div_table.find('thead')

# Lấy tất cả tag <th> ra
ls_th = table_head.find_all('th')

# Lấy ra tên column bằng method 'get_text'
column_names = [th.get_text().replace('\n','').strip() for th in ls_th]

# check lại tên column
column_names

['Mã', 'Công ty', 'Ngành', 'Sàn', 'Khối lượng NY/ĐKGD']

### Step 3: Lấy data cho các rows.

In [ ]:
# tìm tag "tbody" và lưu vào 1 biến
tbody = div_table.find('tbody')

# tìm tất cả các row có trong này
ls_tr = tbody.find_all('tr')

# khởi tạo list trống chứa nội dung các row
ls_row_value = []

# tạo vòng lặp để lấy data trong từng cell(ô) của từng row và lưu vào list
for tr in ls_tr:
  ls_td = tr.find_all('td')
  ls_value = [td.get_text() for td in ls_td]
  ls_row_value.append(ls_value)

# kiểm tra lại nội dung các rows
ls_row_value

[['AAA', 'Công ty cổ phần Nhựa An Phát Xanh', 'Nhựa, cao su & sợi', 'HOSE']]

> Kết quả trên rõ ràng bị thiếu quá nhiều so với bảng chúng ta thấy trong website ban đầu.

> Nguyên nhân thì Hv có đọc thêm tìm hiểu và cho là do trong file dạng html ban đầu gửi đi từ server không có tag 'tbody' mà do browser tự động thêm vào dẫn đến tình trạng trên. (Nếu Hv hiểu sai thì nhờ Ban Giám Khảo - Super Teacher- Mentor cho ý kiến nếu được).

In [ ]:
# tìm tag 'tr' thay vì tag 'tbody' và lưu kết quả vào 1 biến
ls_tr = soup.find_all('tr')

# thực hiện tuần tự các bước như ô code phía trên
ls_row_value = []
for tr in ls_tr:
  ls_td = tr.find_all("td")
  ls_value = [td.get_text() for td in ls_td]
  ls_row_value.append(ls_value)

# kiểm tra lại nội dung các rows
ls_row_value

[[],
 ['AAA', 'Công ty cổ phần Nhựa An Phát Xanh', 'Nhựa, cao su & sợi', 'HOSE'],
 ['AAM',
  'Công ty Cổ phần Thủy sản Mekong',
  'Nuôi trồng nông & hải sản',
  'HOSE'],
 ['AAT', 'Công ty cổ phần Tiên Sơn Thanh Hoá', 'Hàng May mặc', 'HOSE'],
 ['ABR',
  'Công ty Cổ phần Đầu tư Nhãn hiệu Việt',
  'Tư vấn & Hỗ trợ KD',
  'HOSE'],
 ['ABS',
  'Công ty Cổ phần Dịch vụ Nông nghiệp Bình Thuận',
  'Phân phối hàng chuyên dụng',
  'HOSE'],
 ['ABT',
  'Công ty Cổ phần Xuất nhập khẩu thủy sản Bến Tre',
  'Nuôi trồng nông & hải sản',
  'HOSE'],
 ['ACB', 'Ngân hàng Thương mại Cổ phần Á Châu', 'Ngân hàng', 'HOSE'],
 ['ACC',
  'Công ty cổ phần Đầu tư và Xây dựng Bình Dương ACC',
  'Vật liệu xây dựng & Nội thất',
  'HOSE',
  '104,999,993'],
 ['ACG',
  'Công ty Cổ phần Gỗ An Cường',
  'Lâm sản và Chế biến gỗ',
  'HOSE',
  '150,787,946'],
 ['ACL',
  'Công ty Cổ phần Xuất nhập khẩu Thủy sản Cửu Long An Giang',
  'Nuôi trồng nông & hải sản',
  'HOSE',
  '50,159,019'],
 ['ADG',
  'Công ty Cổ phần Clever Grou

> Lần này cho ra kết quả nhiều hơn so với lần trước, tuy nhiên 7 row đầu tiên (từ 'AAA' đến 'ACB') vẫn bị thiếu dữ liệu ở column cuối cùng ('Khối lượng NY/ĐKGD').

In [ ]:
# tìm các tag 'td' thay vì 'tr' và lưu kết quả vào 1 biến
ls_td = soup.find_all('td')

 # khởi tạo list trống chứa nội dung các row
ls_row_value = []

# khởi tạo list trống chứa nội dung 1 row trong vòng lặp
row = []

# tạo vòng lặp lấy ra data trong 5 ô (vì table trong web có 5 cột) và lưu vào 1 row
for td in ls_td:
  row.append(td.get_text())
  if len(row) == 5:
    ls_row_value.append(row) # lưu row này vào list chứa nội dung các row
    row = [] # trả về 0 để lưu data của row sau đó

# check lại các hàng trong bảng
ls_row_value

[['AAA',
  'Công ty cổ phần Nhựa An Phát Xanh',
  'Nhựa, cao su & sợi',
  'HOSE',
  '382,274,496'],
 ['AAM',
  'Công ty Cổ phần Thủy sản Mekong',
  'Nuôi trồng nông & hải sản',
  'HOSE',
  '12,346,411'],
 ['AAT',
  'Công ty cổ phần Tiên Sơn Thanh Hoá',
  'Hàng May mặc',
  'HOSE',
  '70,819,103'],
 ['ABR',
  'Công ty Cổ phần Đầu tư Nhãn hiệu Việt',
  'Tư vấn & Hỗ trợ KD',
  'HOSE',
  '20,000,000'],
 ['ABS',
  'Công ty Cổ phần Dịch vụ Nông nghiệp Bình Thuận',
  'Phân phối hàng chuyên dụng',
  'HOSE',
  '80,000,000'],
 ['ABT',
  'Công ty Cổ phần Xuất nhập khẩu thủy sản Bến Tre',
  'Nuôi trồng nông & hải sản',
  'HOSE',
  '14,387,207'],
 ['ACB',
  'Ngân hàng Thương mại Cổ phần Á Châu',
  'Ngân hàng',
  'HOSE',
  '3,884,050,358'],
 ['ACC',
  'Công ty cổ phần Đầu tư và Xây dựng Bình Dương ACC',
  'Vật liệu xây dựng & Nội thất',
  'HOSE',
  '104,999,993'],
 ['ACG',
  'Công ty Cổ phần Gỗ An Cường',
  'Lâm sản và Chế biến gỗ',
  'HOSE',
  '150,787,946'],
 ['ACL',
  'Công ty Cổ phần Xuất nhập kh

> Lần này đã cho đầy đủ kết quả nên ta tiến hành qua bước tiếp theo.

### Step 4: Lưu data vào 1 DataFrame để kiểm tra.

In [ ]:
# lưu data vào DataFrame
df = pd.DataFrame(ls_row_value, columns = column_names)

# xem qua 5 dòng đầu
df.head()

,Mã,Công ty,Ngành,Sàn,Khối lượng NY/ĐKGD
0,AAA,Công ty cổ phần Nhựa An Phát Xanh,"Nhựa, cao su & sợi",HOSE,"382,274,496"
1,AAM,Công ty Cổ phần Thủy sản Mekong,Nuôi trồng nông & hải sản,HOSE,"12,346,411"
2,AAT,Công ty cổ phần Tiên Sơn Thanh Hoá,Hàng May mặc,HOSE,"70,819,103"
3,ABR,Công ty Cổ phần Đầu tư Nhãn hiệu Việt,Tư vấn & Hỗ trợ KD,HOSE,"20,000,000"
4,ABS,Công ty Cổ phần Dịch vụ Nông nghiệp Bình Thuận,Phân phối hàng chuyên dụng,HOSE,"80,000,000"


In [ ]:
# kiểm tra thông tin chung
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Mã                  20 non-null     object
 1   Công ty             20 non-null     object
 2   Ngành               20 non-null     object
 3   Sàn                 20 non-null     object
 4   Khối lượng NY/ĐKGD  20 non-null     object
dtypes: object(5)
memory usage: 928.0+ bytes


> Nhận xét:

Thông tin trả về đầy đủ so với table trong website, tuy nhiên column cuối có dạng `text`.

##2/ Cách 2: Dùng `read_html`.

### Step 1: Lấy content của website.


In [ ]:
# địa chỉ website (chỉ thực hiện với trang số 1 trong tổng số 30)
url = "https://24hmoney.vn/companies?industry_code=all&floor_code=HOSE&com_type=all&letter=all&page=1"

# đọc html và lưu vào biến 'tables', encoding là 'utf-8' cho Tiếng Việt
tables = pd.read_html(url, encoding='utf-8')

# kiểm tra số lượng table có trong 'tables'
len(tables)

1

### Step 2: Lưu data vào 1 DataFrame để kiểm tra.

In [ ]:
# lưu data vào biến 'tbl'
tbl = tables[0]

# xem 5 dòng đầu
tbl.head()

,Mã,Công ty,Ngành,Sàn,Khối lượng NY/ĐKGD
0,AAA,Công ty cổ phần Nhựa An Phát Xanh,"Nhựa, cao su & sợi",HOSE,382274496
1,AAM,Công ty Cổ phần Thủy sản Mekong,Nuôi trồng nông & hải sản,HOSE,12346411
2,AAT,Công ty cổ phần Tiên Sơn Thanh Hoá,Hàng May mặc,HOSE,70819103
3,ABR,Công ty Cổ phần Đầu tư Nhãn hiệu Việt,Tư vấn & Hỗ trợ KD,HOSE,20000000
4,ABS,Công ty Cổ phần Dịch vụ Nông nghiệp Bình Thuận,Phân phối hàng chuyên dụng,HOSE,80000000


In [ ]:
# xem các thông tin chung
tbl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Mã                  20 non-null     object
 1   Công ty             20 non-null     object
 2   Ngành               20 non-null     object
 3   Sàn                 20 non-null     object
 4   Khối lượng NY/ĐKGD  20 non-null     int64 
dtypes: int64(1), object(4)
memory usage: 928.0+ bytes


> Nhận xét:

Thông tin trả về đầy đủ so với table trong website và column cuối có dạng `int` đúng với thực tế.

##3/ So sánh
Cách dùng `read_html` tốn ít code và thời gian xử lý hơn đồng thời data cũng đầy đủ hơn và chính xác hơn về kiểu dữ liệu so với `BeautifulSoup` (trong trường hợp này nói riêng và các trường hợp data cần lấy đang ở dạng table trên web nói chung).
> Trong phạm vi Project này sẽ dùng cách `read_html` thay cho `BeautifulSoup`.

#III/ MULTI-PAGES SCRAPING
Sau đây sẽ đi vào tiến hành scrape Web hàng loạt bằng `read_html`.
Đồng thời trong lúc thực hiện sẽ tạo ra **sự tạm nghỉ gián đoạn** giữa các lần đọc url để tránh gây tình trạng truy cập dồn dập đến server làm nghẽn mạng.

##1/ Lấy data.

In [ ]:
# import library để thực hiện sleep ngẫu nhiên
from random import randint
from time import sleep

# tạo dataframe trống để lưu dữ liệu sau khi scrape
HSE_comp = pd.DataFrame()

# tạo vòng lặp để scrape lần lượt cho 30 trang
for i in range(1,31):

  # tạo url dựa vào link + số trang
  url = "https://24hmoney.vn/companies?industry_code=all&floor_code=HOSE&com_type=all&letter=all&page=" + str(i)

  # đọc html và lưu vào list 'tables', encoding là 'utf-8' cho Tiếng Việt
  tables = pd.read_html(url, encoding='utf-8')

  # lưu table trong mỗi trang vào 1 table
  tbl = tables[0]

  # union table mới vào dataframe gốc
  HSE_comp = pd.concat([HSE_comp, tbl])

  # tạo thời gian sleep ngẫu nhiên từ 1-20s:
  s = randint(1,20)

  # viết thông báo thời gian sẽ nghỉ
  print(f'Page {i} done ! Will sleep for {s} seconds')

  # thực hiện sleep
  sleep(s)

# kiểm tra ngẫu nhiên 10 dòng của dataframe kết quả
HSE_comp.sample(10)

Page 1 done ! Will sleep for 18 seconds
Page 2 done ! Will sleep for 2 seconds
Page 3 done ! Will sleep for 13 seconds
Page 4 done ! Will sleep for 7 seconds
Page 5 done ! Will sleep for 12 seconds
Page 6 done ! Will sleep for 2 seconds
Page 7 done ! Will sleep for 11 seconds
Page 8 done ! Will sleep for 17 seconds
Page 9 done ! Will sleep for 1 seconds
Page 10 done ! Will sleep for 12 seconds
Page 11 done ! Will sleep for 9 seconds
Page 12 done ! Will sleep for 3 seconds
Page 13 done ! Will sleep for 7 seconds
Page 14 done ! Will sleep for 20 seconds
Page 15 done ! Will sleep for 5 seconds
Page 16 done ! Will sleep for 1 seconds
Page 17 done ! Will sleep for 17 seconds
Page 18 done ! Will sleep for 15 seconds
Page 19 done ! Will sleep for 11 seconds
Page 20 done ! Will sleep for 14 seconds
Page 21 done ! Will sleep for 13 seconds
Page 22 done ! Will sleep for 7 seconds
Page 23 done ! Will sleep for 5 seconds
Page 24 done ! Will sleep for 8 seconds
Page 25 done ! Will sleep for 9 secon

,Mã,Công ty,Ngành,Sàn,Khối lượng NY/ĐKGD
14,CFPT2304,CQ CFPT2304,-,HOSE,-
11,CTCB2305,CQ CTCB2305,-,HOSE,-
14,NHT,Công ty Cổ phần Sản xuất và Thương mại Nam Hoa,Đồ chơi,HOSE,24028169
10,CPOW2303,CQ CPOW2303,-,HOSE,-
11,ADP,Công ty Cổ phần Sơn Á Đông,Vật liệu xây dựng & Nội thất,HOSE,23039850
6,CHPG2322,CQ CHPG2322,-,HOSE,-
4,CSTB2304,Chung quyen.STB.KIS.M.CA.,-,HOSE,-
13,CVRE2313,CQ CVRE2313,-,HOSE,-
3,CHDB2301,Chung quyen.HDB.KIS.M.CA.,-,HOSE,-
0,RDP,Công ty cổ phần Rạng Đông Holding,"Nhựa, cao su & sợi",HOSE,49069803


In [ ]:
#  kiểm tra 10 dòng cuối
HSE_comp.tail(10)

,Mã,Công ty,Ngành,Sàn,Khối lượng NY/ĐKGD
5,VPS,Công ty Cổ phần thuốc sát trùng Việt Nam,"Sản phẩm hóa dầu, Nông dược & Hóa chất khác",HOSE,24460792
6,VRC,Công ty cổ phần Bất động sản và Đầu tư VRC,Bất động sản,HOSE,50000000
7,VRE,Công ty Cổ phần Vincom Retail,Bất động sản,HOSE,2328818410
8,VSC,Công ty Cổ phần Tập đoàn Container Việt Nam,"Kho bãi, hậu cần và bảo dưỡng",HOSE,133395642
9,VSH,Công ty Cổ phần Thủy điện Vĩnh Sơn - Sông Hinh,Sản xuất & Phân phối Điện,HOSE,236241246
10,VSI,Công ty Cổ phần đầu tư và xây dựng cấp thoát nước,Xây dựng,HOSE,13200000
11,VTB,Công ty Cổ phần Viettronics Tân Bình,Hàng điện & điện tử,HOSE,11982050
12,VTO,Công ty Cổ phần Vận tải xăng dầu VITACO,Vận tải Thủy,HOSE,79866666
13,YBM,Công ty cổ phần Khoáng sản công nghiệp Yên Bái,Khai khoáng,HOSE,14299880
14,YEG,Công ty cổ phần Tập đoàn Yeah1,Giải trí & Truyền thông,HOSE,76279968


> Kết quả 10 dòng cuối khớp với trang 30 của website nên coi như quá trình scraping đã hoàn tất và đạt yêu cầu.

##2/ Xử lý sơ bộ.

In [ ]:
# reset lại index cho dataframe
HSE_comp.reset_index(inplace=True, drop = True)

# kiểm tra thông tin chung
HSE_comp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 595 entries, 0 to 594
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Mã                  595 non-null    object
 1   Công ty             595 non-null    object
 2   Ngành               595 non-null    object
 3   Sàn                 595 non-null    object
 4   Khối lượng NY/ĐKGD  595 non-null    object
dtypes: object(5)
memory usage: 23.4+ KB


> Dataframe có 595 hàng và 5 cột, không có null.
> Phát hiện thấy column `Khối lượng NY/ĐKGD` đang có dạng `text` mặc dù lúc scraping thử với trang 1 là dạng `int` >>> cần kiểm tra thêm.

In [ ]:
# kiểm tra với describe
HSE_comp.describe(include='O')

,Mã,Công ty,Ngành,Sàn,Khối lượng NY/ĐKGD
count,595,595,595,595,595
unique,595,589,62,1,400
top,AAA,Chung quyen.HPG.KIS.M.CA.,-,HOSE,-
freq,1,2,184,595,184


> Hai cột `Ngành` và `Khối lượng NY/ĐKGD` hiện đang có giá trị null dưới dạng `text` là '-' với số lượng 184 mỗi cột.

In [ ]:
# check các row với column 'Ngành' có value là '-'
HSE_comp[HSE_comp['Ngành'] == '-']

,Mã,Công ty,Ngành,Sàn,Khối lượng NY/ĐKGD
45,CACB2302,CQ CACB2302,-,HOSE,-
46,CACB2303,CQ CACB2303,-,HOSE,-
47,CACB2304,CQ CACB2304,-,HOSE,-
52,CFPT2214,CHUNG QUYEN FPT-HSC-MET12,-,HOSE,-
53,CFPT2303,CHUNG QUYEN FPT/ACBS/CALL,-,HOSE,-
...,...,...,...,...,...
250,CVRE2310,CQ CVRE2310,-,HOSE,-
251,CVRE2311,CQ CVRE2311,-,HOSE,-
252,CVRE2312,CQ CVRE2312,-,HOSE,-
253,CVRE2313,CQ CVRE2313,-,HOSE,-


> 184 cột này thể hiện các mã Chứng Quyền (và số lượng này trùng khớp với 184 hàng có value `-` chúng ta thấy lúc xem `info`) không phải **CP thông thường** của Công ty nên chúng ta sẽ tiến hành bỏ đi.

> Tương tự cho các mã có `Ngành` là `Quỹ đầu tư` chúng ta cũng bỏ đi.

In [ ]:
# tiến hành bỏ các row là mã Chứng Quyền
HSE = HSE_comp.drop(HSE_comp[HSE_comp['Ngành'] == '-'].index)

# kiểm tra
HSE.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 411 entries, 0 to 594
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Mã                  411 non-null    object
 1   Công ty             411 non-null    object
 2   Ngành               411 non-null    object
 3   Sàn                 411 non-null    object
 4   Khối lượng NY/ĐKGD  411 non-null    object
dtypes: object(5)
memory usage: 19.3+ KB


In [ ]:
# tiếp tục bỏ các row là mã Quỹ đầu tư
HSE = HSE.drop(HSE_comp[HSE_comp['Ngành'] == 'Quỹ đầu tư'].index)

# kiểm tra
HSE.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 394 entries, 0 to 594
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Mã                  394 non-null    object
 1   Công ty             394 non-null    object
 2   Ngành               394 non-null    object
 3   Sàn                 394 non-null    object
 4   Khối lượng NY/ĐKGD  394 non-null    object
dtypes: object(5)
memory usage: 18.5+ KB


In [ ]:
# bỏ luôn cột 'Sàn' vì trong phạm vi của Project chúng ta chỉ xem xét 1 sàn HOSE
HSE = HSE.drop(columns='Sàn')

# convert dữ liệu column cuối cùng sang dạng 'int'
HSE['Khối lượng NY/ĐKGD']=HSE['Khối lượng NY/ĐKGD'].astype(int)

# kiểm tra
HSE.info()

# xem một số row ngẫu nhiên
HSE.sample(20)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 394 entries, 0 to 594
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Mã                  394 non-null    object
 1   Công ty             394 non-null    object
 2   Ngành               394 non-null    object
 3   Khối lượng NY/ĐKGD  394 non-null    int64 
dtypes: int64(1), object(3)
memory usage: 15.4+ KB


,Mã,Công ty,Ngành,Khối lượng NY/ĐKGD
576,VNL,Công ty Cổ phần LOGISTICS VINALINK,Dịch vụ vận tải,14140487
345,HHS,Công ty Cổ phần Đầu tư Dịch vụ Hoàng Huy,Sản xuất ô tô,347160713
560,VDP,Công ty Cổ phần Dược phẩm Trung ương Vidipha,Dược phẩm,16831904
362,HU1,Công ty Cổ phần Đầu tư và Xây dựng HUD1,Xây dựng,10000000
275,DPG,Công ty Cổ phần Đạt Phương,Xây dựng,62999554
384,KPF,Công ty cổ phần Đầu tư tài chính Hoàng Minh,Vật liệu xây dựng & Nội thất,60867241
516,TDC,Công ty Cổ phần Kinh doanh và Phát triển Bình ...,"Tư Vấn, Định giá, Môi giới Bất động sản",100000000
283,DTA,Công ty Cổ phần Đệ Tam,Bất động sản,18059832
425,OGC,Công ty Cổ phần Tập đoàn Đại Dương,Tài chính đặc biệt,300000000
517,TDG,Công ty Cổ phần Đầu tư TDG GLOBAL,Phân phối xăng dầu & khí đốt,18446831


##3/ Save kết quả vào CSV.

In [ ]:
# lưu file vào Collab dạng csv với tên là 'hse_comp'
HSE.to_csv("/content/B1_hse_comp.csv", index = False, encoding='utf-8')

In [ ]:
# đọc file kiểm tra
pd.read_csv('B1_hse_comp.csv')

,Mã,Công ty,Ngành,Khối lượng NY/ĐKGD
0,AAA,Công ty cổ phần Nhựa An Phát Xanh,"Nhựa, cao su & sợi",382274496
1,AAM,Công ty Cổ phần Thủy sản Mekong,Nuôi trồng nông & hải sản,12346411
2,AAT,Công ty cổ phần Tiên Sơn Thanh Hoá,Hàng May mặc,70819103
3,ABR,Công ty Cổ phần Đầu tư Nhãn hiệu Việt,Tư vấn & Hỗ trợ KD,20000000
4,ABS,Công ty Cổ phần Dịch vụ Nông nghiệp Bình Thuận,Phân phối hàng chuyên dụng,80000000
...,...,...,...,...
389,VSI,Công ty Cổ phần đầu tư và xây dựng cấp thoát nước,Xây dựng,13200000
390,VTB,Công ty Cổ phần Viettronics Tân Bình,Hàng điện & điện tử,11982050
391,VTO,Công ty Cổ phần Vận tải xăng dầu VITACO,Vận tải Thủy,79866666
392,YBM,Công ty cổ phần Khoáng sản công nghiệp Yên Bái,Khai khoáng,14299880


> Tới đây xem như quá trình scraping data lấy thông tin các Công ty đang niêm yết trên sàn HOSE từ website đã hoàn tất.